In [1]:
import pandas as pd
import os 
import os.path as osp
import glob

In [2]:
os.getcwd()

'D:\\Project- Electro\\social_behavior_electro\\data_organization\\python'

In [3]:
# folder_name = 'D://Project- Electro//social_behavior_electro//analysis//affiliative//population analysis results'
# sociability = 'affiliative'
# sociability = 'aversive'
sociability = 'free'

folder_name = osp.join('..','..','analysis',sociability,'population analysis results')
df_dict_list = glob.glob(osp.join(folder_name, 'Population analysis results for LFP - Mean power*'))
df_dict_list

['..\\..\\analysis\\free\\population analysis results\\Population analysis results for LFP - Mean power_free_111_2.xlsx',
 '..\\..\\analysis\\free\\population analysis results\\Population analysis results for LFP - Mean power_free_14_16.xlsx']

In [4]:
def fix_file_paths(df_files):
    
    df_files = df_files.rename(columns={df_files.columns[0]:'timestamps', df_files.columns[0]:'filename', })
    
    return df_files


In [7]:
# # Get number of session per group
sheet_name = 'Uploaded files'
# df_files = fix_file_paths(df_dict_list[sheet_name])
# max_len = len(df_files)

In [11]:
df_temp

,values,area,stage,freq,filenames
0,NaN,16,3_After,30-80,free_Rat1-probe4-day2-Behavior_and_Optogenetic...
1,3.7367,16,3_After,30-80,free_Rat2-probe1-Sniffing-day1-Behavior_and_Op...
2,2.4670,16,3_After,30-80,Free_Rat2-probe1-sniffing-day2-Behavior_and_Op...
3,3.8709,16,3_After,30-80,free_Rat2-probe1-sniffing-day3-Behavior_and_Op...
4,6.7207,16,3_After,30-80,Free_Rat3-probe2-Sniffing-Day1-Behavior_and_Op...
...,...,...,...,...,...
57,NaN,16,3_After,30-80,Free_Rat22-probe21-Day1-Behavior_and_Optogenet...
58,NaN,16,3_After,30-80,Free_Rat23-probe22-Day1-Behavior_and_Optogenet...
59,NaN,16,3_After,30-80,Free_Rat23-probe22-Day2-Behavior_and_Optogenet...
60,NaN,16,3_After,30-80,Free_Rat26-Probe25-Day1-Behavior_and_Optogenet...


In [8]:
substrings_to_remove = ['Mean','Before','After', 'During','First', 'Second', 'Area']

df_all_list = []
for df_name in df_dict_list:
    df_dict = pd.read_excel(df_name, None)
    
    file_sheet_name = 'Uploaded files'
    filenames = fix_file_paths(df_dict[sheet_name])
    filenames = filenames. iloc[:,0]
    max_len = len(filenames)
    
    

    for sheet, df in df_dict.items():
#         if len(df.columns) > 1:
#             continue
        if "All" in sheet:
            continue
            
        if "Uploaded" in sheet:
            continue
        
        
        df_temp = pd.DataFrame()
        area_name = sheet
        for sub_string_to_remove in substrings_to_remove:
            area_name = area_name.replace(sub_string_to_remove,'')
        area_name = str(area_name)

        vals = df.iloc[:max_len]
        
        if "Before" in sheet:
            stage = "1_Before"
        elif "During" in sheet:
            stage = "2_During"
        elif "After" in sheet:
            stage = "3_After"
        else:
            print(sheet)
            raise
            
        if "First" in sheet:
            freq = '4-12'
        elif "Second" in sheet:
            freq = "30-80"
        else:
            raise
        
        df_temp['values'] = vals
        df_temp['area'] = area_name
        df_temp['stage'] = stage
        df_temp['freq'] = freq
        df_temp['filenames'] = filenames
        
        df_all_list.append(df_temp)
df_all = pd.concat(df_all_list)          

In [9]:
df = df_all.copy()
# Create a pivot table to unstack 'freq' and 'stage' columns
pivot_df = pd.pivot_table(df, values='values', index=['filenames'], columns=['area','freq','stage'])

# Reset the index to flatten the columns
pivot_df = pivot_df.reset_index()
pivot_df = pivot_df.set_index(('filenames',"",""))
pivot_df = pivot_df.rename_axis(index = {('filenames',"",""):'filenames'})

In [10]:
pivot_df


area                                                    111           \
freq                                                  30-80            
stage                                              1_Before 2_During   
filenames                                                              
Free_Rat16-probe16-day1-Behavior_and_Optogeneti...   1.0991   3.7972   
Free_Rat16-probe16-day2-Behavior_and_Optogeneti...   2.6020   5.1779   
Free_Rat16-probe16-day4-Behavior_and_Optogeneti...   3.8735   5.2641   
Free_Rat17-Probe14-day1-Behavior_and_Optogeneti...   7.6919  10.2044   
Free_Rat17-probe14-day2-Behavior_and_Optogeneti...   7.0441   8.4373   
...                                                     ...      ...   
free_Rat8-probe12-day3_Behavior_and_Optogenetic...   1.3016   2.2237   
free_Rat9-probe11-day1_Behavior_and_Optogenetic...  11.4753  13.0846   
free_Rat9-probe11-day2_Behavior_and_Optogenetic...  11.8992  13.2167   
free_Rat9-probe11-day3_Behavior_and_Optogenetic...  12.3076  14.4574   
free_Rat9-probe11-day4_Behavior_and_Optogenetic...  12.5065  14.2306   

area                                                                   \
freq                                                             4-12   
stage                                                3_After 1_Before   
filenames                                                               
Free_Rat16-probe16-day1-Behavior_and_Optogeneti...   0.76132  16.6417   
Free_Rat16-probe16-day2-Behavior_and_Optogeneti...   3.42660  18.1176   
Free_Rat16-probe16-day4-Behavior_and_Optogeneti...   3.35620  19.6168   
Free_Rat17-Probe14-day1-Behavior_and_Optogeneti...   8.43490  21.1246   
Free_Rat17-probe14-day2-Behavior_and_Optogeneti...   7.25960  20.3642   
...                                                      ...      ...   
free_Rat8-probe12-day3_Behavior_and_Optogenetic...   1.37310  16.4948   
free_Rat9-probe11-day1_Behavior_and_Optogenetic...  11.42420  23.4687   
free_Rat9-probe11-day2_Behavior_and_Optogenetic...  11.62700  24.0709   
free_Rat9-probe11-day3_Behavior_and_Optogenetic...  12.63700  24.0036   
free_Rat9-probe11-day4_Behavior_and_Optogenetic...  13.20200  24.0251   

area                                                                       14  \
freq                                                                    30-80   
stage                                              2_During  3_After 1_Before   
filenames                                                                       
Free_Rat16-probe16-day1-Behavior_and_Optogeneti...  18.6605  16.9729      NaN   
Free_Rat16-probe16-day2-Behavior_and_Optogeneti...  21.3028  19.1129      NaN   
Free_Rat16-probe16-day4-Behavior_and_Optogeneti...  22.9356  19.1687      NaN   
Free_Rat17-Probe14-day1-Behavior_and_Optogeneti...  21.0480  21.1528  12.0088   
Free_Rat17-probe14-day2-Behavior_and_Optogeneti...  20.8815  20.3157  10.4607   
...                                                     ...      ...      ...   
free_Rat8-probe12-day3_Behavior_and_Optogenetic...  19.4427  16.0873      NaN   
free_Rat9-probe11-day1_Behavior_and_Optogenetic...  24.6213  23.1986      NaN   
free_Rat9-probe11-day2_Behavior_and_Optogenetic...  25.2711  24.2186      NaN   
free_Rat9-probe11-day3_Behavior_and_Optogenetic...  26.2026  24.4976      NaN   
free_Rat9-probe11-day4_Behavior_and_Optogenetic...  26.0971  24.3250      NaN   

area                                                                           \
freq                                                                     4-12   
stage                                              2_During  3_After 1_Before   
filenames                                                                       
Free_Rat16-probe16-day1-Behavior_and_Optogeneti...      NaN      NaN      NaN   
Free_Rat16-probe16-day2-Behavior_and_Optogeneti...      NaN      NaN      NaN   
Free_Rat16-probe16-day4-Behavior_and_Optogeneti...      NaN      NaN      NaN   
Free_Rat17-Probe14-day1-Behavior_and_Opt

In [25]:
# Function to calculate the differences and add columns
def add_difference_columns(df):
    areas = df.columns.get_level_values('area').unique()
    freqs = df.columns.get_level_values('freq').unique()

    for area in areas:
        for freq in freqs:
            
            before_col = (area, freq, '1_Before')
            during_col = (area, freq, '2_During')
            after_col = (area, freq, '3_After')
            
            diff_during_before = df[during_col] - df[before_col]
            diff_after_before = df[after_col] - df[before_col]

            diff_col_name_during_before = (area,freq,'4_DiffDuringBefore')
            diff_col_name_after_before =  (area,freq,'5_DiffAfterBefore')

            df[diff_col_name_during_before] = diff_during_before
            df[diff_col_name_after_before] = diff_after_before

    return df

# Call the function to add difference columns
df_with_diff = add_difference_columns(pivot_df)


In [26]:
def sort_columns(df):
    # Sort the columns based on the specified rule
    sorted_columns = sorted(df.columns, key=lambda col: (col[0], col[1], col[2]))

    # Reindex the DataFrame with the sorted columns
    sorted_df = df.reindex(columns=sorted_columns)

    return sorted_df


df_with_diff_sorted = sort_columns(df_with_diff)
df_with_diff_sorted.columns

MultiIndex([('111', '30-80',           '1_Before'),
            ('111', '30-80',           '2_During'),
            ('111', '30-80',            '3_After'),
            ('111', '30-80', '4_DiffDuringBefore'),
            ('111', '30-80',  '5_DiffAfterBefore'),
            ('111',  '4-12',           '1_Before'),
            ('111',  '4-12',           '2_During'),
            ('111',  '4-12',            '3_After'),
            ('111',  '4-12', '4_DiffDuringBefore'),
            ('111',  '4-12',  '5_DiffAfterBefore'),
            ( '14', '30-80',           '1_Before'),
            ( '14', '30-80',           '2_During'),
            ( '14', '30-80',            '3_After'),
            ( '14', '30-80', '4_DiffDuringBefore'),
            ( '14', '30-80',  '5_DiffAfterBefore'),
            ( '14',  '4-12',           '1_Before'),
            ( '14',  '4-12',           '2_During'),
            ( '14',  '4-12',            '3_After'),
            ( '14',  '4-12', '4_DiffDuringBefore'),
            

In [27]:
areas = df_with_diff_sorted.columns.get_level_values('area').unique()
areas

Index(['111', '14', '16', '2'], dtype='object', name='area')

In [28]:
# save files
df_with_diff_sorted.to_excel(osp.join(folder_name,f'lfp_rawdata_{sociability}.xlsx'))